In [120]:
# make the screen bigger!

from IPython.display import display, HTML

display(HTML(data="""   <style>     div#notebook-container    { width: 100%; }     div#menubar-container     { width: 85%; }     div#maintoolbar-container { width: 99%; } </style>
"""))

# import all our libraries (if this cell throws an error you may need to reinstall a library)
import os
import pandas as pd
import shutil   
import re
from datetime import date, datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import stats

%matplotlib notebook


### Import custom functions

In [121]:
from datetime import date, datetime, timedelta

%run Functions/Funcs.py

In [138]:
pristine_raw_data       = os.path.join('..', 'Raw_data/Weather_stations/Raw_data')
Raw_data_copy  = os.path.join('.', 'Workspace/Raw_data_copy')
Formatted_data = os.path.join('.', 'Workspace/Formatted_raw_data')

# string to look for is anything with "time" in it for taking out trash headings
p = re.compile('time', re.IGNORECASE) 
# Final data headings
headings = ['DateTime', 'SRD_wpm2', 'HMD_pct', 'TMP_F', 'RNF_in', 'WND_deg', 'WNS_mph']


# make a copy of the raw data
files1 = os.listdir(pristine_raw_data) # make a list of all the files in the directory
for f in files1:
    shutil.copy(os.path.join(pristine_raw_data, f), Raw_data_copy)
    
# Change any dat files to csv     (they are just csvs it looks like so rename them locally)
for i in os.listdir(Raw_data_copy):
    if i.endswith('.dat'):
        nam = os.path.splitext(i)[0]
        os.rename(os.path.join(Raw_data_copy, i), os.path.join(Raw_data_copy, nam+".csv"))
        print("Converting {} to {}.csv".format(i, nam))


# Process each raw file 
for i in os.listdir(Raw_data_copy): 
    
    # For the excells
    if i.endswith('.xlsx'):
        thefile = os.path.join(Raw_data_copy, i)
        # clean the header and read in  the data  (using one of custom functions
        data = xls_cleaner(thefile)              
        nam = i.split(".")[0]                    # remove the .xls
        
        # save to same filder but as a csv now.
        data.to_csv(os.path.join(Raw_data_copy, nam+'.csv'), encoding='utf-8') # note: this step is where duplicate files will dissapear as they will be named the exact same thing and then ocerwitten by the loop 
        print("Converting {} to {}.csv".format(i, nam))

Converting Aasu-CR300wx.dat to Aasu-CR300wx.csv
Converting Afono-wx-060519.dat to Afono-wx-060519.csv
Converting Poloa-CR300wx.dat to Poloa-CR300wx.csv
Converting Vaipito-CR300wx.dat to Vaipito-CR300wx.csv
Converting Aasu -Weather-0616161.xlsx to Aasu -Weather-0616161.csv
Converting Aasu -Weather-0616162.xlsx to Aasu -Weather-0616162.csv
Converting Aasu Downloaded01182016.xlsx to Aasu Downloaded01182016.csv
Converting Aasu Wx 020917.xlsx to Aasu Wx 020917.csv
Converting Aasu Wx 092816.xlsx to Aasu Wx 092816.csv
Converting Aasu Wx 2016 & Notes (2).xlsx to Aasu Wx 2016 & Notes (2).csv
Converting Aasu Wx 2016 & Notes (3).xlsx to Aasu Wx 2016 & Notes (3).csv
Converting Aasu Wx 2016 & Notes.xlsx to Aasu Wx 2016 & Notes.csv
Converting Aasu _Downloaded10072015.xlsx to Aasu _Downloaded10072015.csv
Converting Aasu _Wx_071916.xlsx to Aasu _Wx_071916.csv
Converting Aasu-Wx-013018.xlsx to Aasu-Wx-013018.csv
Converting Afono-Wx-013018.xlsx to Afono-Wx-013018.csv
Converting Afono-Wx-102418.xlsx to A

In [146]:
# Process CSV files

p = re.compile('time', re.IGNORECASE)  # string to look for is anything with "time" in it for taking out trash headings
headings = ['DateTime', 'SRD_wpm2', 'HMD_pct', 'TMP_F', 'RNF_in', 'WND_deg', 'WNS_mph']  # Final data headings


# Process each csv file 
for i in files1:        
    # For the csvs
    if i.endswith('.csv'):
        tempdata_good =pd.DataFrame(columns = headings)                                  # blank dataframe to write to

        with open(os.path.join(Raw_data_copy, i), 'r') as f_in:                 # open file 
            text_line_data = f_in.read().splitlines(True)                       # temporary object for finding header

        for skips in range(0,6):
            matcher = p.search(text_line_data[skips])                           # find the row that has the word time in it
            if matcher:
                rows2skip = skips                                               # ID the number of rows to skip to make the time row the header row
        tempdata = pd.read_csv(os.path.join(Raw_data_copy, i), skiprows=rows2skip)  # read in dataset, with the correct header

        tempdata.columns = tempdata.columns.str.strip()                         # remove any whitespace in the collumn headdings 

    # this block standardizes all the columns to the same names
        for col in tempdata.columns:
            if col in ("TIMESTAMP","Timestamp", "Date and Time" ):
                tempdata =  tempdata.rename(columns={col: "DateTime"} )
            if col in ("TMP" , "Air Temp (F)", "AirTF_Avg"):
                tempdata =  tempdata.rename(columns={col: "TMP_F"} )
            if col in ("HMD", "RH", "Relative Humidity"):
                tempdata =  tempdata.rename(columns={col: "HMD_pct"} )
            if col in ("SRD", "Solar Rad (W_AVG)", "SlrMJ_Tot"):
                tempdata =  tempdata.rename(columns={col: "SRD_wpm2"} )
            if col in ("RNF", "Rain_in_Tot", "Rain (in)"):
                tempdata =  tempdata.rename(columns={col: "RNF_in"} )
            if col in ("WND", "WindDir_D1_WVT", "Wind Direction (D1)"):
                tempdata =  tempdata.rename(columns={col: "WND_deg"} )       
            if col in ("WNS", "Wind Speed (MPH)", "WS_mph_S_WVT"):
                tempdata =  tempdata.rename(columns={col: "WNS_mph"} )  

    # This block takes out unwanted headings and resets index
        try:
            for col2 in headings:
                tempdata_good[col2] = tempdata[col2]                                    # select only the headings we want
        except:                                                    # this except pass keeps it from hanging up where there are missing columns like in the ASPA SR station   
            pass
        tempdata_good = tempdata_good.reset_index(drop=True)                       # reset the row index to start from 0

    #this names the data key based on its first and last date (if there is a repeat file it will not make a repeat data entry)
        i = i.replace(" ", "_")
        i = i.replace("-", "_")                # this takes any dashes and underscores turns them to spaces so that the first word can be used for the name even is there is a dash

        tempdata_good['DateTime'] = pd.to_datetime(tempdata_good['DateTime'], errors='coerce')  # string date to real date
    # note: some of the time stamps from the stations are not perfectily on 15 mins, this rounds to the nearest 15    

        tempdata_good['DateTime'] = tempdata_good['DateTime'].apply(lambda dt: datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))

        nam = i.split("_")[0] +'_'+ str(tempdata_good['DateTime'].iloc[0]).split(" ")[0] +'_to_'+str(tempdata_good['DateTime'].iloc[-1]).split(" ")[0]
        
        data = tempdata_good.copy()
            
        data.to_csv(os.path.join(Formatted_data, nam+'.csv'), encoding='utf-8') # note: this step is where duplicate files will dissapear as they will be named the exact same thing and then ocerwitten by the loop 
        print("Cleaning {}.csv".format(nam))
        
        


    

Cleaning Aasu_2015-07-21_to_2016-01-19.csv
Cleaning Aasu_2016-12-15_to_2017-01-07.csv
Cleaning Aasu_2016-08-12_to_2016-09-28.csv
Cleaning Aasu_2016-01-01_to_2016-08-12.csv
Cleaning Aasu_2016-01-01_to_2016-11-28.csv
Cleaning Aasu_2016-01-01_to_2016-12-15.csv
Cleaning Aasu_2015-07-21_to_2015-10-06.csv
Cleaning Aasu_2016-06-08_to_2016-07-18.csv
Cleaning Aasu_2017-04-25_to_2018-01-29.csv
Cleaning Alava_2017-07-01_to_2018-01-10.csv
Cleaning Alava_2016-12-20_to_2017-01-19.csv
Cleaning Alava_2017-05-01_to_2017-08-02.csv
Cleaning Alava_2016-08-16_to_2016-09-20.csv
Cleaning Alava_2016-01-01_to_2016-11-29.csv
Cleaning Alava_2016-01-01_to_2016-12-20.csv
Cleaning Alava_2017-01-01_to_2017-06-07.csv
Cleaning Alava_2017-05-01_to_2017-10-11.csv
Cleaning Alava_2015-07-30_to_2015-10-08.csv
Cleaning ASPA_2016-08-18_to_2016-10-06.csv
Cleaning ASPA_2016-05-26_to_2016-06-07.csv
Cleaning ASPA_2016-04-08_to_2016-08-18.csv
Cleaning ASPA_2016-04-08_to_2016-11-30.csv
Cleaning ASPA_2016-04-08_to_2016-12-19.csv
Cl

In [ ]:
def xls_cleaner(file_2_read, datetime_name = ["Date and Time", "Timestamp",  "TIMESTAMP"] ):

    data_xls = pd.read_excel(file_2_read,  header=None, index_col=None)                        # read  xlsx initially with no cols or index
    data_xls_trimmed = data_xls.apply(lambda x: x.str.strip() if x.dtype == "object" else x)   # remove whitespace from all strigs
    Col_true = data_xls_trimmed[data_xls_trimmed.isin(datetime_name)]                          # find the cell that has the start of the header   Note the datetime col has to always be ahead of data columns
    Col_true = Col_true.dropna(how='all')                                                      # retain and select the index and col numbers of the cell with the start of the header
    finally_frame = Col_true.dropna(axis = 1, how='all')
    rows_to_skip = finally_frame.index[0]
    index_col_number = finally_frame.columns[0]

    data = pd.read_excel(file_2_read,  skiprows=rows_to_skip,  index_col=index_col_number)  # read  xlsx to memory

    return data
